In [ ]:
!git clone https://github.com/matterport/Mask_RCNN.git

In [ ]:
import yaml
import numpy as np
import cv2
import mrcnn.config
import mrcnn.utils
from mrcnn.model import MaskRCNN
import matplotlib as pt

In [ ]:
root = '/parking/'
model_path = os.path.join(root, "logs")
coco_path =  os.path.join(root, "mask_rcnn_coco.h5")
video_path = "/parking/FTUI_parking_spot.mp4"

In [ ]:
class MaskRCNNConfig(mrcnn.config.Config):
    NAME = "coco_pretrained_model_config"
    IMAGES_PER_GPU = 1
    GPU_COUNT = 1
    NUM_CLASSES = 1 + 80  
    DETECTION_MIN_CONFIDENCE = 0.5

In [ ]:
model = MaskRCNN(mode="inference", model_dir=model_path, config=MaskRCNNConfig())
model.load_weights(coco_path, by_name=True)
video_cv = cv2.VideoCapture(video_path)

In [ ]:
parking_spot_yaml = "spots_v5.yml"
parking_spots=[]

with open(parking_spot_yaml, 'r') as stream:
    temp_yml_park_spot = yaml.load(stream)

if temp_yml_park_spot != None:
    for spot in temp_yml_park_spot:
      coords = np.array(park['points'])
      parking_spots.append(coords)

In [ ]:
def cars_coords(coords, ids):
    bounding_box = []
    for i, x in enumerate(coords):
        if ids[i] == 3:  #id 3 represent car
            bounding_box.append(x)

    return np.array(bounding_box)

In [ ]:
#run video frame by frame
counter = 0
while video_cv.isOpened():
  counter += 1
  _, frame = video_capture.read()
  
  
  #if counter%10 == 0:
   #     print("FRANE" + str(counter))
   #     pt.pyplot.imshow(frame2)
    #    pt.pyplot.show()



  #convert to RGB
  frame_rgb = frame[:,:,::-1]
  hasil = model.detect([frame_rgb])
  
  detected_car_boundingbox = cars_coords(hasil[0]['rois'], hasil[0]['class_ids'])


  if counter%8 == 0 :
    overlaps = mrcnn.utils.compute_overlaps(parking_spots, detected_car_boundingbox)
    spot_count=0
    spot_check = [] #store empty or filled spot data
    for spot, overlap in zip(parking_spots, overlaps):
      print("its spot", spot_count)
      
      temp_ratio = np.max(overlap_areas)
      if temp_ratio < 0.2 :           #if the overlap ratio below threshold then its consider empty
        y1, x1, y2, x2 = spot
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)    #set color to green
        spot_check.append(1)
      else:
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 1)     #if not set to red
        spot_check.append(0)
      spot_count +=1
    

video_capture.release()
